# 2章　パーセプトロン

2入力のパーセプトロンで AND NAND OR ゲートを実装する  
動作原理

\begin{cases} 
    0 \quad ( w_{1}x_{1} + w_{2}x_{2} <=　{θ}) \\
    1 \quad ( w_{1}x_{1} + w_{2}x_{2} >　{θ}) \\
            \end{cases} 
      
w1 w2 θ の値でゲートを作用させる


In [1]:
def AND(x1, x2):
    w1, w2, theta = 0.5, 0.5, 0.7
    tmp = x1*w1 + x2*w2
    if tmp <= theta:
        return 0
    elif tmp > theta:
        return 1

In [11]:
AND(0, 0)

0

In [12]:
AND(1, 0)

0

In [13]:
AND(0, 1)

0

In [14]:
AND(1, 1)

1

In [9]:
import numpy as np

In [10]:
def AND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.7
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    elif tmp > 0:
        return 1

In [15]:
def NAND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([-0.5, -0.5])
    b = 0.7
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    elif tmp > 0:
        return 1

In [19]:
def OR(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.2
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    elif tmp > 0:
        return 1

In [20]:
def XOR(x1, x2):
    s1 = NAND(x1, x2)
    s2 = OR(x1, x2)
    y = AND(s1, s2)
    return y

# 3章　活性化関数

In [23]:
def step_function(x):
    if x > 0:
        return 1
    else:
        return 0

In [24]:
def step_function(x):
    y = x > 0
    # bool to int 
    return y.astype(np.int)